# EXP09: Generalization Verification (Routing + Worst-group)
**목표**: EXP08 EDA에서 도출한 아키텍처 가설(라우팅 + 조건부 파이프라인)을 실제 성능/안정성 지표로 검증

### 배경
- EXP01~07은 단일 문서 최적화 중심으로 성능 개선을 달성했으나, EXP08에서 코퍼스 이질성(테이블/이미지/포맷)과 대표성 문제를 확인
- 특히 고려대 PDF는 전체 코퍼스 대비 극단값(97~99 백분위)으로, 단일 설정의 일반화 한계가 확인됨
- 따라서 Exp09는 “파라미터 미세튜닝”이 아니라 “문서군 라우팅 아키텍처” 검증을 목적으로 함

### 실험 구조
| Phase | 내용 | 목적 |
|-------|------|------|
| Phase 0 | Dataset Split & Fingerprint | 중앙군/극단군 분해, 라우팅 입력 특징 고정 |
| Phase 1 | Routing Policy 정의 | 결정론적 라우팅 + 불확실 구간 다중 경로 정책 수립 |
| Phase 2 | Routing Ablation (A/B/C) | 단일 vs 라우팅 vs 보수적 다중 라우팅 비교 |
| Phase 3 | Table/Image 2x2 Ablation | table-aware/OCR 기여도를 subgroup 단위로 분해 |
| Phase 4 | 운영 안정성 + 최종 선택 | 품질-지연-성공률 trade-off 기반 최종 의사결정 |

### 핵심 KPI
| KPI | 정의 |
|-----|------|
| Overall Mean | 전체 문서 평균 성능 |
| Macro(Group Mean) | 그룹별 평균의 평균 |
| Worst-group | 가장 낮은 그룹 성능 |
| Ops KPI | ingestion success, timeout rate, fallback rate, p95 latency |



In [1]:
# ============================================================
# 1. Setup
# ============================================================
import os
import re
import json
import time
from pathlib import Path
from datetime import datetime
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

SEED = 42
np.random.seed(SEED)

DATA_DIR = Path('../data/raw/files')
EXP_DIR = Path('../data/experiments')
OUT_DIR = EXP_DIR
OUT_DIR.mkdir(parents=True, exist_ok=True)

EDA_REPORT = EXP_DIR / 'exp08_eda_report.json'
EDA_RESULTS = EXP_DIR / 'exp08_eda_results.csv'
OUT_REPORT = OUT_DIR / 'exp09_report.json'
OUT_RESULTS = OUT_DIR / 'exp09_results.csv'

print('[Setup] ready')
print('  DATA_DIR:', DATA_DIR)
print('  EDA_REPORT exists:', EDA_REPORT.exists())
print('  EDA_RESULTS exists:', EDA_RESULTS.exists())


[Setup] ready
  DATA_DIR: ..\data\raw\files
  EDA_REPORT exists: True
  EDA_RESULTS exists: True


In [2]:
# ============================================================
# 2. Data Load
# ============================================================
if EDA_RESULTS.exists():
    df = pd.read_csv(EDA_RESULTS)
else:
    rows = []
    for p in sorted(DATA_DIR.glob('*')):
        if p.is_file():
            rows.append({'file': p.name, 'format': p.suffix.lower().lstrip('.')})
    df = pd.DataFrame(rows)

print(f'[Data] documents: {len(df)}')
print(df.head(3))

[Data] documents: 100
                                                file format  file_size_kb  \
0       (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp    hwp        4163.0   
1  (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원...    hwp         330.0   
2         (사）한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.hwp    hwp         881.0   

   text_len  n_tables  n_images extract_method error  
0    104685       226         9       hwp5html   NaN  
1     51341       107        17       hwp5html   NaN  
2     14407         0         6        hwp5txt   NaN  


In [3]:
# ============================================================
# 3. Helper Functions
# ============================================================
def safe_col(df, name, default=0):
    if name in df.columns:
        return df[name].fillna(default)
    return pd.Series([default] * len(df), index=df.index)


def build_fingerprint(df: pd.DataFrame) -> pd.DataFrame:
    x = df.copy()
    x['text_len'] = safe_col(x, 'text_len', 0)
    x['n_tables'] = safe_col(x, 'n_tables', 0)
    x['n_images'] = safe_col(x, 'n_images', 0)
    x['file_size_kb'] = safe_col(x, 'file_size_kb', 0)
    x['error_flag'] = safe_col(x, 'error', '').astype(str).str.len().gt(0).astype(int)

    # 복합 난이도 스코어 (가중치는 Phase 0에서 조정 가능)
    z = lambda s: (s - s.mean()) / (s.std() + 1e-9)
    x['difficulty_score'] = (
        0.35 * z(x['text_len'])
        + 0.30 * z(x['n_tables'])
        + 0.20 * z(x['n_images'])
        + 0.15 * z(x['file_size_kb'])
        + 0.50 * x['error_flag']
    )
    return x


def split_groups(fp: pd.DataFrame, q=0.8) -> pd.DataFrame:
    y = fp.copy()
    thr = y['difficulty_score'].quantile(q)
    y['group'] = np.where(y['difficulty_score'] >= thr, 'extreme', 'central')
    return y


def route_policy(row, low=0.45, high=0.70):
    # 결정론적 규칙 + 불확실 구간 다중 경로
    table_density = row.get('n_tables', 0)
    image_density = row.get('n_images', 0)
    fmt = str(row.get('format', '')).lower()

    score_table = 0.0
    score_image = 0.0

    if table_density >= 80:
        score_table += 0.55
    if table_density >= 150:
        score_table += 0.25
    if image_density >= 20:
        score_image += 0.55
    if image_density >= 80:
        score_image += 0.25
    if fmt == 'pdf':
        score_table += 0.10
        score_image += 0.10

    best = max(score_table, score_image)
    if best < low:
        return {'route': 'text_only', 'route_mode': 'single', 'confidence': round(best, 3)}
    if low <= best < high:
        # 불확실: 다중 경로
        if score_table >= score_image:
            return {'route': 'text+table', 'route_mode': 'multi', 'confidence': round(best, 3)}
        return {'route': 'text+image', 'route_mode': 'multi', 'confidence': round(best, 3)}

    if score_table >= score_image:
        return {'route': 'table_aware', 'route_mode': 'single', 'confidence': round(best, 3)}
    return {'route': 'image_aware', 'route_mode': 'single', 'confidence': round(best, 3)}

---
## Phase 0: Dataset Split & Fingerprint
**실험**: 문서 fingerprint 생성 후 복합 난이도 기준으로 중앙군/극단군 분할
- fingerprint: format, file_size_kb, text_len, n_tables, n_images, error_flag
- difficulty_score 기반 quantile split (기본 q=0.8)
- 결과를 `doc_fingerprint` 테이블로 저장


In [4]:
# ============================================================
# Phase 0: Fingerprint 생성 및 그룹 분할
# ============================================================
fp = build_fingerprint(df)
fp = split_groups(fp, q=0.8)

print('[Phase 0] group distribution')
print(fp['group'].value_counts())
print('[Phase 0] difficulty score summary')
print(fp['difficulty_score'].describe())

show_cols = [c for c in ['file', 'format', 'file_size_kb', 'text_len', 'n_tables', 'n_images', 'difficulty_score', 'group'] if c in fp.columns]
print('[Phase 0] sample')
print(fp[show_cols].head(10))

[Phase 0] group distribution
group
central    80
extreme    20
Name: count, dtype: int64
[Phase 0] difficulty score summary
count    100.000000
mean       0.015000
std        0.772988
min       -1.174810
25%       -0.315434
50%        0.016695
75%        0.296969
max        5.022243
Name: difficulty_score, dtype: float64
[Phase 0] sample
                                                file format  file_size_kb  \
0       (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp    hwp        4163.0   
1  (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원...    hwp         330.0   
2         (사）한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.hwp    hwp         881.0   
3                 (재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.hwp    hwp        1138.5   
4       2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.hwp    hwp         235.5   
5       BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).hwp    hwp         808.0   
6  KOICA 전자조달_[긴급] [지문] [국제] 우즈베키스탄 열린 의정활동 상하원 .hwp    hwp       23546.0   
7          경기도 안양시_호계체육관 배드민턴장 및 탁구장 예약시스템 구

## Phase 0 결과 해석

### Fingerprint 기반 그룹 분할 요약

| 항목 | 수치 | 의미 |
|------|------|------|
| 전체 문서 수 | 100 | EXP08 코퍼스 전체 사용 |
| 그룹 분할 | central 80 / extreme 20 | q=0.8 기준으로 상위 20%를 극단군으로 분리 |
| difficulty_score | mean 0.015 / std 0.773 | 점수 분포가 중앙 0 부근에 형성 |
| difficulty_score 범위 | min -1.175 / Q1 -0.315 / median 0.017 / Q3 0.297 / max 5.022 | 우측 꼬리가 긴 분포, 소수 고난도 문서 존재 |

### 핵심 발견

**1. 중앙군/극단군 분할이 의도대로 작동했다**
- 80:20 비율로 그룹이 안정적으로 분리되어, 이후 worst-group 평가 기반이 확보되었다.

**2. 난이도 분포는 long-tail 특성을 보인다**
- 최대값(5.022)이 Q3(0.297) 대비 매우 커서, 일부 문서가 난이도를 크게 끌어올리는 구조다.
- 이는 EXP08에서 확인한 “극단 문서가 전체 성능 해석을 왜곡”하는 위험과 일치한다.

**3. 극단군은 단일 특성이 아닌 복합 특성으로 형성된다**
- 샘플에서 extreme 문서는 `장문+고테이블` 또는 `고이미지` 유형이 함께 관측된다.
- 즉, 극단군 정의를 단일 지표가 아니라 복합 점수로 둔 현재 방식이 타당하다.

### Phase 1 시사점

1. **라우팅 정책의 1차 검증 대상은 extreme 20건**으로 설정해야 한다.
2. **route 로그(route/route_mode/confidence)**를 그룹별로 분리 집계해, 극단군에서 오분기율이 증가하는지 확인해야 한다.
3. 불확실 구간(중간 confidence)은 Phase 2에서 다중 라우팅(C안)의 실질 이득을 검증할 핵심 구간이다.
4. 이후 성능 비교는 overall 평균뿐 아니라 **macro(group mean) + worst-group**을 기본 리포트로 고정한다.

---
## Phase 1: Routing Policy 정의
**실험**: 결정론적 규칙 기반 라우팅 + 불확실 구간 다중 경로 정책 정의
- 출력 로그: route, route_mode(single/multi), confidence
- 목표: 재현 가능성(동일 입력 동일 경로) + 디버깅 가능성(경로 근거 기록)


In [5]:
# ============================================================
# Phase 1: 라우팅 정책 적용
# ============================================================
routes = fp.apply(lambda r: pd.Series(route_policy(r)), axis=1)
phase1_df = pd.concat([fp.reset_index(drop=True), routes.reset_index(drop=True)], axis=1)

print('[Phase 1] route distribution')
print(phase1_df['route'].value_counts())
print('[Phase 1] route mode distribution')
print(phase1_df['route_mode'].value_counts())
print('[Phase 1] confidence summary')
print(phase1_df['confidence'].describe())

[Phase 1] route distribution
route
text+table     48
text_only      31
table_aware    19
image_aware     1
text+image      1
Name: count, dtype: int64
[Phase 1] route mode distribution
route_mode
single    51
multi     49
Name: count, dtype: int64
[Phase 1] confidence summary
count    100.000000
mean       0.433500
std        0.309125
min        0.000000
25%        0.000000
50%        0.550000
75%        0.550000
max        0.900000
Name: confidence, dtype: float64


In [6]:
# ============================================================
# Phase 1: 라우팅 로그 검증
# ============================================================
log_cols = [c for c in ['file', 'group', 'route', 'route_mode', 'confidence', 'difficulty_score'] if c in phase1_df.columns]
route_log = phase1_df[log_cols].copy()
print(route_log.head(15))

                                                 file    group        route  \
0        (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp  extreme  table_aware   
1   (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원...  central   text+table   
2          (사）한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.hwp  central    text_only   
3                  (재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.hwp  central   text+table   
4        2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.hwp  central   text+table   
5        BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).hwp  central   text+table   
6   KOICA 전자조달_[긴급] [지문] [국제] 우즈베키스탄 열린 의정활동 상하원 .hwp  extreme  image_aware   
7           경기도 안양시_호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역.hwp  central   text+table   
8                  경기도사회서비스원_2024년 통합사회정보시스템 운영지원.hwp  central   text+table   
9             경기도평택시_2024년도 평택시 버스정보시스템(BIS) 구축사업.hwp  central   text+table   
10          경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).hwp  central   text+table   
11            경희대학교_[입찰공고] 산학협력단 정보시스템 운영 용역업체 선정.hw

## Phase 1 결과 해석

### 라우팅 정책 적용 요약

| 항목 | 수치 | 의미 |
|------|------|------|
| route 분포 | text+table 48 / text_only 31 / table_aware 19 / image_aware 1 / text+image 1 | table 신호가 강한 문서가 다수이며, image 단
독 경로는 소수 |
| route_mode 분포 | single 51 / multi 49 | 단일/다중 경로가 거의 1:1로 분배 |
| confidence | mean 0.434 / median 0.55 / min 0.00 / max 0.90 | 0.55 근처 중간 신뢰 구간이 크게 형성됨 |

### 핵심 발견

**1. 정책이 “조건부 라우팅” 의도를 충실히 반영했다**
- `text_only(31)` + `table_aware(19)` + `image_aware(1)` + `text+table(48)` + `text+image(1)`로 경로가 다양하게 분기됨.
- 특히 table 관련 경로(`text+table + table_aware`)가 67건으로, 코퍼스의 테이블 밀집 특성과 정합적이다.

**2. 불확실 구간 다중 경로가 실제로 많이 활성화된다**
- `multi=49%`로 절반 수준이며, 다중 경로가 예외 처리 수준이 아니라 핵심 운영 모드에 가깝다.
- 이는 Phase 2에서 C안(규칙+다중 라우팅)의 실효성을 검증할 필요성을 강화한다.

**3. confidence 분포가 이산적(0.00/0.55/0.80~0.90 중심)이다**
- median이 0.55이고 25%가 0.00인 구조는 규칙 기반 스코어가 몇 개 임계값에 집중되어 있음을 시사한다.
- 현재 단계에선 정상 동작으로 보이며, 이후 성능 차이가 미미하면 confidence calibration을 후속 과제로 다루면 된다.

### Phase 2 시사점

1. **A/B/C 비교를 group별로 필수 분해**: 전체 평균뿐 아니라 central/extreme 각각에서 성능을 보고해야 한다.
2. **worst-group을 핵심 KPI로 고정**: 라우팅의 목적은 평균 상승보다 극단군 붕괴 방지에 있음.
3. **multi 경로 비용 검증 필요**: C안의 품질 이득이 latency/timeout 증가를 상쇄하는지 함께 판단해야 한다.
4. **table 경로 기여도 확인 우선**: route 분포상 table 관련 경로 비중이 크므로, Phase 3 2x2에서 table-aware 효과를 먼저 검증한다.


---
## Phase 2: Routing Ablation (A/B/C)
**실험**: 라우팅 아키텍처 형태별 성능 비교
- A: 단일 파이프라인(기존)
- B: 규칙 기반 단일 라우팅
- C: 규칙 + 불확실 구간 다중 라우팅(2경로 실행 후 rerank)
- 지표: KW_v2, Faithfulness, CR, Latency + group/macro/worst-group


In [7]:
# ============================================================
# Phase 2: 실험 매트릭스 정의 + 실행 계획 생성
# ============================================================
ABLATION_CONFIGS = [
    {'config': 'A_single_pipeline', 'routing': 'off', 'multi_route': 'off'},
    {'config': 'B_rule_single_route', 'routing': 'on', 'multi_route': 'off'},
    {'config': 'C_rule_multi_route', 'routing': 'on', 'multi_route': 'on'},
]

ablation_df = pd.DataFrame(ABLATION_CONFIGS)
print(ablation_df)

# 결과 스키마 (실측치 파일)
result_schema_cols = [
    'config', 'run_id', 'file', 'group',
    'kw_v2', 'faithfulness', 'context_recall', 'latency_sec',
    'ingestion_success', 'timeout_rate', 'fallback_rate', 'p95_latency_sec'
]
print('\n[Expected metrics schema]')
print(result_schema_cols)

# -----------------------------------------------------------------
# 1) Config별 문서 실행 계획 생성 (phase1_df 필요)
# -----------------------------------------------------------------
if 'phase1_df' not in locals():
    raise RuntimeError('phase1_df가 없습니다. Phase 1을 먼저 실행하세요.')


def effective_route_for_config(route: str, route_mode: str, cfg_name: str):
    """A/B/C 설정에 따라 실제 실행 경로를 결정"""
    r = str(route)
    m = str(route_mode)

    if cfg_name == 'A_single_pipeline':
        return 'text_only', 'single'

    if cfg_name == 'B_rule_single_route':
        if r == 'text+table':
            return 'table_aware', 'single'
        if r == 'text+image':
            return 'image_aware', 'single'
        return r, 'single'

    # C_rule_multi_route
    return r, m


route_cost = {
    'text_only': 1.0,
    'table_aware': 1.25,
    'image_aware': 1.35,
    'text+table': 1.55,
    'text+image': 1.65,
}


def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))


plan_rows = []
for _, row in phase1_df.iterrows():
    for cfg in ABLATION_CONFIGS:
        cfg_name = cfg['config']
        eff_route, eff_mode = effective_route_for_config(row['route'], row['route_mode'], cfg_name)

        diff = float(row.get('difficulty_score', 0.0))
        conf = float(row.get('confidence', 0.0))
        base = route_cost.get(eff_route, 1.0)

        # deterministic ops proxy (실행 전 dry-run)
        latency_sec = 18 + 10 * base + 9 * max(diff, 0)
        timeout_rate = float(np.clip(sigmoid(-2.3 + 0.75 * base + 0.90 * max(diff, 0)), 0.01, 0.85))
        fallback_rate = float(np.clip(0.02 + 0.08 * (1 if eff_mode == 'multi' else 0) + 0.12 * timeout_rate + 0.05 * (1 - conf), 0.01, 0.95))
        ingestion_success = float(np.clip(1 - timeout_rate, 0.0, 1.0))

        plan_rows.append({
            'config': cfg_name,
            'file': row.get('file', None),
            'group': row.get('group', 'unknown'),
            'route': row.get('route', None),
            'route_mode': row.get('route_mode', None),
            'effective_route': eff_route,
            'effective_mode': eff_mode,
            'confidence': conf,
            'difficulty_score': diff,
            'latency_sec': float(latency_sec),
            'ingestion_success': ingestion_success,
            'timeout_rate': timeout_rate,
            'fallback_rate': fallback_rate,
        })

phase2_plan_df = pd.DataFrame(plan_rows)

# config/group별 p95 latency 계산
p95_df = phase2_plan_df.groupby(['config', 'group'])['latency_sec'].quantile(0.95).reset_index(name='p95_latency_sec')
phase2_plan_df = phase2_plan_df.merge(p95_df, on=['config', 'group'], how='left')

PLAN_PATH = OUT_DIR / 'exp09_phase2_plan.csv'
phase2_plan_df.to_csv(PLAN_PATH, index=False, encoding='utf-8-sig')

print('\n[Phase 2] execution plan generated')
print('  rows:', len(phase2_plan_df))
print('  saved:', PLAN_PATH)
print('\n[Plan distribution: config x group]')
print(phase2_plan_df.groupby(['config', 'group']).size().unstack(fill_value=0))

# -----------------------------------------------------------------
# 2) 실측 결과 파일 로드/템플릿 자동 생성
# -----------------------------------------------------------------
METRICS_PATH = OUT_DIR / 'exp09_phase2_metrics.csv'
METRICS_TEMPLATE_PATH = OUT_DIR / 'exp09_phase2_metrics_template.csv'
phase2_metrics_df = pd.DataFrame(columns=result_schema_cols)

# 템플릿(문서별 A/B/C 1회 run 기본) 자동 생성
template_df = phase2_plan_df[['config', 'file', 'group', 'latency_sec', 'ingestion_success', 'timeout_rate', 'fallback_rate', 'p95_latency_sec']].copy()
template_df.insert(1, 'run_id', 1)
template_df['kw_v2'] = np.nan
template_df['faithfulness'] = np.nan
template_df['context_recall'] = np.nan
template_df = template_df[result_schema_cols]
template_df.to_csv(METRICS_TEMPLATE_PATH, index=False, encoding='utf-8-sig')
print('\n[Phase 2] metrics template saved')
print('  template:', METRICS_TEMPLATE_PATH)

# 실측 파일이 없으면 초기 skeleton 파일을 자동 생성(덮어쓰기 금지)
if METRICS_PATH.exists():
    raw = pd.read_csv(METRICS_PATH)
    missing = [c for c in result_schema_cols if c not in raw.columns]
    if missing:
        print('\n[WARN] metrics 파일 컬럼 부족:', missing)
        print('       템플릿 파일(exp09_phase2_metrics_template.csv)을 기준으로 수정하세요.')
    else:
        phase2_metrics_df = raw[result_schema_cols].copy()
        print('\n[Phase 2] measured metrics loaded')
        print('  rows:', len(phase2_metrics_df))
        print('  from:', METRICS_PATH)
else:
    template_df.to_csv(METRICS_PATH, index=False, encoding='utf-8-sig')
    phase2_metrics_df = template_df.copy()
    print('\n[INFO] exp09_phase2_metrics.csv가 없어 skeleton을 생성했습니다.')
    print('       kw_v2 / faithfulness / context_recall 컬럼만 채우고 다시 실행하면 됩니다.')
    print('  created:', METRICS_PATH)

# 유효 실측행 개수 체크(quality 3개 모두 입력된 행)
valid_mask = phase2_metrics_df[['kw_v2', 'faithfulness', 'context_recall']].notna().all(axis=1) if len(phase2_metrics_df) > 0 else pd.Series(dtype=bool)
phase2_metrics_valid_df = phase2_metrics_df[valid_mask].copy() if len(phase2_metrics_df) > 0 else pd.DataFrame(columns=result_schema_cols)
print('\n[Phase 2] measured quality rows:', len(phase2_metrics_valid_df), '/', len(phase2_metrics_df))

                config routing multi_route
0    A_single_pipeline     off         off
1  B_rule_single_route      on         off
2   C_rule_multi_route      on          on

[Expected metrics schema]
['config', 'run_id', 'file', 'group', 'kw_v2', 'faithfulness', 'context_recall', 'latency_sec', 'ingestion_success', 'timeout_rate', 'fallback_rate', 'p95_latency_sec']

[Phase 2] execution plan generated
  rows: 300
  saved: ..\data\experiments\exp09_phase2_plan.csv

[Plan distribution: config x group]
group                central  extreme
config                               
A_single_pipeline         80       20
B_rule_single_route       80       20
C_rule_multi_route        80       20

[Phase 2] metrics template saved
  template: ..\data\experiments\exp09_phase2_metrics_template.csv

[Phase 2] measured metrics loaded
  rows: 300
  from: ..\data\experiments\exp09_phase2_metrics.csv

[Phase 2] measured quality rows: 186 / 300


### Phase 2-A: A/B/C별 answer/retrieved_contexts 자동 생성

같은 `question/ground_truth`를 유지하고, config별로 서로 다른 출력(`answer`, `retrieved_contexts`)을 생성합니다.

원칙:
- 질문/정답은 고정
- A/B/C는 route/effective_route에 따라 context 선택 전략을 다르게 적용

주의:
- 이 셀은 실험용 자동 생성기입니다(운영 체인과 1:1 동일하지 않음)
- 기존 값을 덮어쓸지 여부는 `OVERWRITE_*` 플래그로 제어합니다.


In [8]:
# ============================================================
# Phase 2-A: A/B/C별 answer/retrieved_contexts 자동 생성
# ============================================================
import subprocess
import re

RAW_EVAL_PATH = OUT_DIR / 'exp09_phase2_raw_eval.csv'
METRICS_PATH = OUT_DIR / 'exp09_phase2_metrics.csv'
if not RAW_EVAL_PATH.exists():
    raise FileNotFoundError(f'raw eval 파일이 없습니다: {RAW_EVAL_PATH}')
if 'phase2_plan_df' not in locals():
    raise RuntimeError('phase2_plan_df가 없습니다. Phase 2 cell 12를 먼저 실행하세요.')

# 덮어쓰기 정책
OVERWRITE_RETRIEVED_CONTEXTS = True
OVERWRITE_ANSWER = True
# Run all 일관성을 위해 생성된 키의 기존 metric 무효화
INVALIDATE_METRICS_ON_GENERATE = True

raw_df = pd.read_csv(RAW_EVAL_PATH)

# phase2_plan_df에는 run_id가 없을 수 있으므로 보정
plan_df = phase2_plan_df.copy()
if 'run_id' not in plan_df.columns:
    plan_df['run_id'] = 1
if 'run_id' not in raw_df.columns:
    raw_df['run_id'] = 1

plan_cols = ['config', 'run_id', 'file', 'group', 'effective_route', 'effective_mode', 'confidence']
plan_key = plan_df[plan_cols].drop_duplicates(['config', 'run_id', 'file', 'group']).copy()
raw_df = raw_df.merge(plan_key, on=['config', 'run_id', 'file', 'group'], how='left')


def is_blank(v):
    if v is None:
        return True
    if isinstance(v, float) and np.isnan(v):
        return True
    s = str(v).strip().lower()
    return s in ('', 'nan', 'none', 'null')


def tok(s):
    if is_blank(s):
        return []
    x = str(s).lower()
    x = re.sub(r"[^0-9a-zA-Z가-힣 ]", " ", x)
    return [t for t in x.split() if len(t) >= 2]


def extract_pdf_text(path: Path):
    try:
        import pdfplumber
        texts = []
        with pdfplumber.open(path) as pdf:
            for p in pdf.pages:
                t = p.extract_text() or ''
                if t.strip():
                    texts.append(t)
        return chr(10).join(texts)
    except Exception:
        return ''


def extract_hwp_text(path: Path):
    try:
        cp = subprocess.run(
            ['hwp5txt', str(path)], capture_output=True, text=True,
            timeout=35, encoding='utf-8', errors='ignore'
        )
        if cp.returncode == 0 and cp.stdout:
            return cp.stdout
        return ''
    except Exception:
        return ''


def split_paragraphs(text):
    if is_blank(text):
        return []
    paras = [p.strip() for p in re.split(r"\n{2,}", text) if p.strip()]
    if len(paras) < 5:
        paras = [ln.strip() for ln in text.split(chr(10)) if ln.strip()]
    out = []
    for p in paras:
        if len(p) <= 1000:
            out.append(p)
        else:
            for i in range(0, len(p), 1000):
                out.append(p[i:i+1000])
    return out


def route_bias_score(paragraph, route):
    ptxt = str(paragraph)
    table_kw = ['표', '배점', '점수', '항목', '금액', '예산', '%', '비율', '기간']
    image_kw = ['그림', '도식', '화면', 'ui', '이미지', '프로세스', '흐름도']

    bias = 0.0
    if route in ('table_aware', 'text+table'):
        bias += 0.30 * sum(1 for k in table_kw if k in ptxt)
    if route in ('image_aware', 'text+image'):
        bias += 0.30 * sum(1 for k in image_kw if k in ptxt.lower())
    return bias


def top_contexts(question, paras, route, mode):
    q = set(tok(question))
    scored = []
    for para in paras:
        pt = set(tok(para))
        if not pt:
            continue
        inter = len(q & pt)
        base = inter / max(1, len(q)) if len(q) > 0 else 0.0
        score = base + route_bias_score(para, route)
        if score > 0:
            scored.append((score, para))
    scored.sort(key=lambda x: x[0], reverse=True)

    if mode == 'multi':
        n = 4
    elif route == 'text_only':
        n = 2
    elif route in ('table_aware', 'image_aware'):
        n = 3
    else:
        n = 3

    return [p for _, p in scored[:n]]


def make_answer(question, contexts):
    if not contexts:
        return '해당 정보를 찾을 수 없습니다.'
    q = set(tok(question))
    best = None
    best_score = -1
    for c in contexts:
        sents = re.split(r'[.!?。]\s+|\n', c)
        for s in sents:
            s = s.strip()
            if len(s) < 6:
                continue
            sc = len(q & set(tok(s)))
            if sc > best_score:
                best_score = sc
                best = s
    if is_blank(best):
        best = str(contexts[0])[:180]
    return str(best).strip()


text_cache = {}
filled_ctx = 0
filled_ans = 0
touched_keys = set()

for i, row in raw_df.iterrows():
    fname = row.get('file')
    question = row.get('question')
    if is_blank(fname) or is_blank(question):
        continue

    do_ctx = OVERWRITE_RETRIEVED_CONTEXTS or is_blank(row.get('retrieved_contexts'))
    do_ans = OVERWRITE_ANSWER or is_blank(row.get('answer'))
    if not do_ctx and not do_ans:
        continue

    fpath = DATA_DIR / str(fname)
    if not fpath.exists():
        continue

    if fname not in text_cache:
        if fpath.suffix.lower() == '.pdf':
            text_cache[fname] = extract_pdf_text(fpath)
        elif fpath.suffix.lower() == '.hwp':
            text_cache[fname] = extract_hwp_text(fpath)
        else:
            text_cache[fname] = ''

    full_text = text_cache.get(fname, '')
    paras = split_paragraphs(full_text)
    if len(paras) == 0:
        continue

    route = row.get('effective_route', 'text_only')
    mode = row.get('effective_mode', 'single')
    ctx = top_contexts(question, paras, route, mode)

    if do_ctx:
        raw_df.at[i, 'retrieved_contexts'] = json.dumps(ctx, ensure_ascii=False)
        filled_ctx += 1
    if do_ans:
        raw_df.at[i, 'answer'] = make_answer(question, ctx)
        filled_ans += 1

    touched_keys.add((row.get('config'), row.get('run_id'), row.get('file'), row.get('group')))

# A/B/C 차이 확인
pivot_ans = raw_df.pivot_table(index=['run_id','file','group','question','ground_truth'], columns='config', values='answer', aggfunc='first')
ratio_ans = np.nan
if all(c in pivot_ans.columns for c in ['A_single_pipeline','B_rule_single_route','C_rule_multi_route']):
    same = (pivot_ans['A_single_pipeline'].fillna('') == pivot_ans['B_rule_single_route'].fillna('')) &            (pivot_ans['B_rule_single_route'].fillna('') == pivot_ans['C_rule_multi_route'].fillna(''))
    ratio_ans = float(same.mean()) if len(same) > 0 else np.nan

pivot_ctx = raw_df.pivot_table(index=['run_id','file','group','question'], columns='config', values='retrieved_contexts', aggfunc='first')
ratio_ctx = np.nan
if all(c in pivot_ctx.columns for c in ['A_single_pipeline','B_rule_single_route','C_rule_multi_route']):
    samec = (pivot_ctx['A_single_pipeline'].fillna('') == pivot_ctx['B_rule_single_route'].fillna('')) &             (pivot_ctx['B_rule_single_route'].fillna('') == pivot_ctx['C_rule_multi_route'].fillna(''))
    ratio_ctx = float(samec.mean()) if len(samec) > 0 else np.nan

# 생성된 키의 metrics 무효화 (Run all 안정화)
invalidated = 0
if INVALIDATE_METRICS_ON_GENERATE and len(touched_keys) > 0 and METRICS_PATH.exists():
    mdf = pd.read_csv(METRICS_PATH)
    key_df = pd.DataFrame(list(touched_keys), columns=['config','run_id','file','group'])
    mdf = mdf.merge(key_df.assign(_touch=1), on=['config','run_id','file','group'], how='left')
    mask = mdf['_touch'].fillna(0).astype(int) == 1
    invalidated = int(mask.sum())
    for c in ['kw_v2','faithfulness','context_recall']:
        if c in mdf.columns:
            mdf.loc[mask, c] = np.nan
    mdf = mdf.drop(columns=['_touch'])
    mdf.to_csv(METRICS_PATH, index=False, encoding='utf-8-sig')

for c in ['effective_route','effective_mode','confidence']:
    if c in raw_df.columns:
        raw_df = raw_df.drop(columns=[c])

raw_df.to_csv(RAW_EVAL_PATH, index=False, encoding='utf-8-sig')
print('[Phase 2-A] saved:', RAW_EVAL_PATH)
print('  filled contexts:', filled_ctx)
print('  filled answers :', filled_ans)
print('  invalidated metric keys:', invalidated)
print('  identical answer ratio (A/B/C):', ratio_ans)
print('  identical context ratio (A/B/C):', ratio_ctx)
print(raw_df[['config','run_id','file','group','question','answer']].head(6))




[Phase 2-A] saved: ..\data\experiments\exp09_phase2_raw_eval.csv
  filled contexts: 294
  filled answers : 294
  invalidated metric keys: 294
  identical answer ratio (A/B/C): 0.43
  identical context ratio (A/B/C): 0.29591836734693877
                config  run_id  \
0    A_single_pipeline       1   
1  B_rule_single_route       1   
2   C_rule_multi_route       1   
3    A_single_pipeline       1   
4  B_rule_single_route       1   
5   C_rule_multi_route       1   

                                                file    group  \
0       (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp  extreme   
1       (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp  extreme   
2       (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp  extreme   
3  (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원...  central   
4  (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원...  central   
5  (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원...  central   

                  question                                           

### Phase 2-B: 실측치 자동 채움 (KW_v2 / Faithfulness / Context Recall)

**입력 파일**: `../data/experiments/exp09_phase2_raw_eval.csv`

- 필수 컬럼: `config, run_id, file, group, question, ground_truth, answer`
- 권장 컬럼(둘 중 하나):
  - `faithfulness, context_recall` (이미 계산된 값)
  - `retrieved_contexts` (RAGAS 자동 계산 시도)

실행 결과:
- `exp09_phase2_metrics.csv`의 `kw_v2/faithfulness/context_recall` 자동 업데이트
- 누락/오류 행은 경고 출력


In [9]:
# ============================================================
# Phase 2-B: 실측치 자동 채움
# ============================================================
import ast
import re
from typing import List

RAW_EVAL_PATH = OUT_DIR / 'exp09_phase2_raw_eval.csv'
METRICS_PATH = OUT_DIR / 'exp09_phase2_metrics.csv'


def is_blank(v) -> bool:
    if v is None:
        return True
    if isinstance(v, float) and np.isnan(v):
        return True
    s = str(v).strip().lower()
    return s in ('', 'nan', 'none', 'null')


def normalize_answer_v2(text: str) -> str:
    if is_blank(text):
        return ''
    s = str(text).lower().strip()
    s = re.sub(r"[\n\t\r]", " ", s)
    s = s.replace('"', ' ').replace("'", ' ')
    s = re.sub(r"[,./()\[\]{}:;|_\-]", " ", s)
    s = ' '.join(s.split())
    return s


def keyword_accuracy_norm_v2(answer: str, ground_truth: str) -> float:
    ans = normalize_answer_v2(answer)
    gt = normalize_answer_v2(ground_truth)
    if not ans or not gt:
        return np.nan
    gt_tokens = [t for t in gt.split(' ') if t]
    if not gt_tokens:
        return np.nan
    ans_set = set([t for t in ans.split(' ') if t])
    hit = sum(1 for t in gt_tokens if t in ans_set)
    return float(hit / len(gt_tokens))


def parse_contexts(v) -> List[str]:
    if is_blank(v):
        return []
    if isinstance(v, list):
        return [str(x) for x in v]
    s = str(v).strip()
    try:
        obj = json.loads(s)
        if isinstance(obj, list):
            return [str(x) for x in obj]
    except Exception:
        pass
    try:
        obj = ast.literal_eval(s)
        if isinstance(obj, list):
            return [str(x) for x in obj]
    except Exception:
        pass
    return [s]


if not METRICS_PATH.exists():
    raise FileNotFoundError(f'metrics 파일이 없습니다: {METRICS_PATH} (Phase 2 cell 12 먼저 실행)')

if not RAW_EVAL_PATH.exists():
    raise FileNotFoundError(
        f'raw eval 파일이 없습니다: {RAW_EVAL_PATH}\n'
        '필수 컬럼: config,run_id,file,group,question,ground_truth,answer'
    )

metrics_df = pd.read_csv(METRICS_PATH)
raw_df = pd.read_csv(RAW_EVAL_PATH)

required_cols = ['config', 'run_id', 'file', 'group', 'question', 'ground_truth', 'answer']
missing = [c for c in required_cols if c not in raw_df.columns]
if missing:
    raise ValueError(f'raw eval 필수 컬럼 누락: {missing}')

agg_cols = ['config', 'run_id', 'file', 'group']

# 기존 metrics 상태와 조인해서 "이미 3개 지표가 모두 있는 키"는 계산 단계부터 skip
base_keys = raw_df[agg_cols].drop_duplicates().copy()
if len(metrics_df) > 0:
    mcols = [c for c in ['kw_v2', 'faithfulness', 'context_recall'] if c in metrics_df.columns]
    mkey = metrics_df[agg_cols + mcols].drop_duplicates().copy()
    base_keys = base_keys.merge(mkey, on=agg_cols, how='left')
else:
    base_keys['kw_v2'] = np.nan
    base_keys['faithfulness'] = np.nan
    base_keys['context_recall'] = np.nan

base_keys['has_all_metrics'] = base_keys[['kw_v2','faithfulness','context_recall']].notna().all(axis=1)
keys_to_compute = base_keys[~base_keys['has_all_metrics']][agg_cols].copy()

# 유효 QA 행 + 계산 필요 키만 대상
valid_qa_mask = (~raw_df['question'].apply(is_blank)) & (~raw_df['ground_truth'].apply(is_blank)) & (~raw_df['answer'].apply(is_blank))
valid_raw_df = raw_df[valid_qa_mask].copy()
if len(keys_to_compute) > 0:
    valid_raw_df = valid_raw_df.merge(keys_to_compute, on=agg_cols, how='inner')
else:
    valid_raw_df = valid_raw_df.iloc[0:0].copy()

print(f'[Phase 2-B] valid QA rows (total): {int(valid_qa_mask.sum())} / {len(raw_df)}')
print(f'[Phase 2-B] keys to compute: {len(keys_to_compute)} / {len(base_keys)}')
print(f'[Phase 2-B] rows to compute: {len(valid_raw_df)}')

if len(valid_raw_df) > 0:
    valid_raw_df['kw_v2_row'] = valid_raw_df.apply(
        lambda r: keyword_accuracy_norm_v2(r.get('answer', ''), r.get('ground_truth', '')), axis=1
    )
    agg = valid_raw_df.groupby(agg_cols, as_index=False).agg(kw_v2=('kw_v2_row', 'mean'))
else:
    agg = pd.DataFrame(columns=agg_cols + ['kw_v2'])

# 분기 조건: 컬럼 존재가 아니라 "실제 값 존재"를 본다
has_faith_cols = 'faithfulness' in raw_df.columns and 'context_recall' in raw_df.columns
has_faith_values = has_faith_cols and valid_raw_df['faithfulness'].notna().any() and valid_raw_df['context_recall'].notna().any()
has_ctx_values = 'retrieved_contexts' in raw_df.columns and valid_raw_df['retrieved_contexts'].apply(lambda x: not is_blank(x)).any()

if has_faith_values:
    fcr_src = valid_raw_df if len(valid_raw_df) > 0 else raw_df.iloc[0:0].copy()
    fcr = fcr_src.groupby(agg_cols, as_index=False).agg(
        faithfulness=('faithfulness', 'mean'),
        context_recall=('context_recall', 'mean')
    )
    agg = agg.merge(fcr, on=agg_cols, how='left') if len(agg) > 0 else fcr
    print('[Phase 2-B] faithfulness/context_recall: raw eval 입력값에서 집계')

elif has_ctx_values and len(valid_raw_df) > 0:
    print('[Phase 2-B] retrieved_contexts 기반으로 RAGAS 계산 시도 (missing keys only)')
    try:
        from datasets import Dataset
        from ragas import evaluate
        from ragas.metrics import Faithfulness, ContextRecall
        from ragas.llms import LangchainLLMWrapper
        from ragas.embeddings import LangchainEmbeddingsWrapper
        from langchain_openai import ChatOpenAI, OpenAIEmbeddings

        class FixedTempChatOpenAI(ChatOpenAI):
            # gpt-5-mini는 temperature=1만 지원하므로 내부 override를 강제
            def _generate(self, messages, stop=None, run_manager=None, **kwargs):
                kwargs['temperature'] = 1
                return super()._generate(messages, stop=stop, run_manager=run_manager, **kwargs)

            async def _agenerate(self, messages, stop=None, run_manager=None, **kwargs):
                kwargs['temperature'] = 1
                return await super()._agenerate(messages, stop=stop, run_manager=run_manager, **kwargs)

        llm = LangchainLLMWrapper(FixedTempChatOpenAI(model='gpt-5-mini', temperature=1, timeout=180, max_retries=3))
        emb = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model='text-embedding-3-small'))

        f_rows = []
        ragas_src = valid_raw_df[valid_raw_df['retrieved_contexts'].apply(lambda x: not is_blank(x))].copy()
        for keys, g in ragas_src.groupby(agg_cols):
            eval_dict = {
                'user_input': g['question'].astype(str).tolist(),
                'response': g['answer'].astype(str).tolist(),
                'retrieved_contexts': g['retrieved_contexts'].apply(parse_contexts).tolist(),
                'reference': g['ground_truth'].astype(str).tolist(),
            }
            ds = Dataset.from_dict(eval_dict)
            ev = evaluate(
                dataset=ds,
                metrics=[Faithfulness(llm=llm), ContextRecall(llm=llm)],
                llm=llm,
                embeddings=emb,
                raise_exceptions=False,
            ).to_pandas()

            f_rows.append({
                'config': keys[0],
                'run_id': keys[1],
                'file': keys[2],
                'group': keys[3],
                'faithfulness': float(ev['faithfulness'].mean()) if 'faithfulness' in ev else np.nan,
                'context_recall': float(ev['context_recall'].mean()) if 'context_recall' in ev else np.nan,
            })

        fcr = pd.DataFrame(f_rows)
        agg = agg.merge(fcr, on=agg_cols, how='left') if len(agg) > 0 else fcr

    except Exception as e:
        print(f'[WARN] RAGAS 계산 실패: {e}')
        if 'faithfulness' not in agg.columns:
            agg['faithfulness'] = np.nan
        if 'context_recall' not in agg.columns:
            agg['context_recall'] = np.nan
else:
    print('[WARN] faithfulness/context_recall 계산 불가')
    print('  - raw eval에 faithfulness/context_recall 실측값을 넣거나, retrieved_contexts를 채우세요.')
    if 'faithfulness' not in agg.columns:
        agg['faithfulness'] = np.nan
    if 'context_recall' not in agg.columns:
        agg['context_recall'] = np.nan

merged = metrics_df.merge(agg, on=agg_cols, how='left', suffixes=('', '_new'))

# 이미 값이 있는 셀은 유지하고, 빈 값(NaN)만 채움
update_stats = {}
for c in ['kw_v2', 'faithfulness', 'context_recall']:
    new_col = f'{c}_new'
    if new_col not in merged.columns:
        continue
    if c not in merged.columns:
        merged[c] = np.nan

    before_na = merged[c].isna().sum()
    fill_mask = merged[c].isna() & merged[new_col].notna()
    merged.loc[fill_mask, c] = merged.loc[fill_mask, new_col]
    after_na = merged[c].isna().sum()
    update_stats[c] = int(before_na - after_na)

drop_cols = [c for c in merged.columns if c.endswith('_new')]
merged = merged.drop(columns=drop_cols)
merged.to_csv(METRICS_PATH, index=False, encoding='utf-8-sig')

valid_mask = merged[['kw_v2', 'faithfulness', 'context_recall']].notna().all(axis=1)
print('[Phase 2-B] metrics 업데이트 완료')
print('  filled cells:', update_stats)
print('  saved:', METRICS_PATH)
print('  valid rows:', int(valid_mask.sum()), '/', len(merged))
print('  preview:')
print(merged[['config', 'run_id', 'file', 'group', 'kw_v2', 'faithfulness', 'context_recall']].head(10))


[Phase 2-B] valid QA rows (total): 300 / 300
[Phase 2-B] keys to compute: 300 / 300
[Phase 2-B] rows to compute: 300
[Phase 2-B] retrieved_contexts 기반으로 RAGAS 계산 시도 (missing keys only)


C:\Users\c\AppData\Local\Temp\ipykernel_60532\2421447912.py:138: DeprecationWarning: Importing Faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import Faithfulness
  from ragas.metrics import Faithfulness, ContextRecall
C:\Users\c\AppData\Local\Temp\ipykernel_60532\2421447912.py:138: DeprecationWarning: Importing ContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import ContextRecall
  from ragas.metrics import Faithfulness, ContextRecall
C:\Users\c\AppData\Local\Temp\ipykernel_60532\2421447912.py:153: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  llm = Langcha

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

[Phase 2-B] metrics 업데이트 완료
  filled cells: {'kw_v2': 294, 'faithfulness': 294, 'context_recall': 294}
  saved: ..\data\experiments\exp09_phase2_metrics.csv
  valid rows: 294 / 300
  preview:
                config  run_id  \
0    A_single_pipeline       1   
1  B_rule_single_route       1   
2   C_rule_multi_route       1   
3    A_single_pipeline       1   
4  B_rule_single_route       1   
5   C_rule_multi_route       1   
6    A_single_pipeline       1   
7  B_rule_single_route       1   
8   C_rule_multi_route       1   
9    A_single_pipeline       1   

                                                file    group  kw_v2  \
0       (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp  extreme    1.0   
1       (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp  extreme    0.0   
2       (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp  extreme    0.0   
3  (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원...  central    0.0   
4  (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원...  central    0.0   
5

In [10]:
# ============================================================
# Phase 2: 집계 (overall/macro/worst-group + ops)
# ============================================================
def summarize_quality(df_metrics: pd.DataFrame):
    if len(df_metrics) == 0:
        return pd.DataFrame(), pd.DataFrame()

    metric_cols = ['kw_v2', 'faithfulness', 'context_recall']
    grp = df_metrics.groupby(['config', 'group'])[metric_cols].mean().reset_index()

    overall = df_metrics.groupby('config')[metric_cols].mean().reset_index()
    macro = grp.groupby('config')[metric_cols].mean().reset_index()
    worst = grp.groupby('config')[metric_cols].min().reset_index()

    overall['view'] = 'overall_mean'
    macro['view'] = 'macro_group_mean'
    worst['view'] = 'worst_group'

    merged = pd.concat([overall, macro, worst], ignore_index=True)
    return grp, merged


def summarize_ops(df_plan: pd.DataFrame):
    ops_cols = ['ingestion_success', 'timeout_rate', 'fallback_rate', 'latency_sec', 'p95_latency_sec']
    grp = df_plan.groupby(['config', 'group'])[ops_cols].mean().reset_index()

    overall = grp.groupby('config')[ops_cols].mean().reset_index()
    macro = grp.groupby('config')[ops_cols].mean().reset_index()
    worst = grp.groupby('config')[['ingestion_success']].min().reset_index()
    worst_timeout = grp.groupby('config')[['timeout_rate', 'fallback_rate', 'latency_sec', 'p95_latency_sec']].max().reset_index()
    worst = worst.merge(worst_timeout, on='config', how='left')

    overall['view'] = 'overall_mean'
    macro['view'] = 'macro_group_mean'
    worst['view'] = 'worst_group'

    merged = pd.concat([overall, macro, worst], ignore_index=True)
    return grp, merged


# 0) metrics를 항상 파일에서 재로딩 (메모리 stale 방지)
METRICS_PATH = OUT_DIR / 'exp09_phase2_metrics.csv'
if METRICS_PATH.exists():
    phase2_metrics_df = pd.read_csv(METRICS_PATH)
else:
    phase2_metrics_df = pd.DataFrame(columns=['config','run_id','file','group','kw_v2','faithfulness','context_recall'])

if len(phase2_metrics_df) > 0:
    valid_mask = phase2_metrics_df[['kw_v2','faithfulness','context_recall']].notna().all(axis=1)
    phase2_metrics_valid_df = phase2_metrics_df[valid_mask].copy()
else:
    phase2_metrics_valid_df = pd.DataFrame()

print('[Phase 2] quality valid rows (reloaded):', len(phase2_metrics_valid_df), '/', len(phase2_metrics_df))

# 1) quality summary (유효 실측치만)
phase2_quality_group = pd.DataFrame()
phase2_quality_view = pd.DataFrame()
if len(phase2_metrics_valid_df) > 0:
    phase2_quality_group, phase2_quality_view = summarize_quality(phase2_metrics_valid_df)
    print('[Phase 2] quality summary by group')
    print(phase2_quality_group)
    print()
    print('[Phase 2] quality summary views')
    print(phase2_quality_view)
else:
    print('[Phase 2] quality metrics not ready')
    print('  - exp09_phase2_metrics.csv의 kw_v2/faithfulness/context_recall를 입력 후 재실행하세요.')

# 2) ops summary (dry-run baseline)
phase2_ops_group, phase2_ops_view = summarize_ops(phase2_plan_df)
print()
print('[Phase 2] ops summary by group (dry-run)')
print(phase2_ops_group)
print()
print('[Phase 2] ops summary views (dry-run)')
print(phase2_ops_view)

# 3) phase2 통합 결과 저장
summary_out = {
    'quality_group_rows': int(len(phase2_quality_group)),
    'quality_view_rows': int(len(phase2_quality_view)),
    'ops_group_rows': int(len(phase2_ops_group)),
    'ops_view_rows': int(len(phase2_ops_view)),
}

PHASE2_OPS_GROUP = OUT_DIR / 'exp09_phase2_ops_group.csv'
PHASE2_OPS_VIEW = OUT_DIR / 'exp09_phase2_ops_view.csv'
phase2_ops_group.to_csv(PHASE2_OPS_GROUP, index=False, encoding='utf-8-sig')
phase2_ops_view.to_csv(PHASE2_OPS_VIEW, index=False, encoding='utf-8-sig')

if len(phase2_quality_group) > 0:
    PHASE2_Q_GROUP = OUT_DIR / 'exp09_phase2_quality_group.csv'
    PHASE2_Q_VIEW = OUT_DIR / 'exp09_phase2_quality_view.csv'
    phase2_quality_group.to_csv(PHASE2_Q_GROUP, index=False, encoding='utf-8-sig')
    phase2_quality_view.to_csv(PHASE2_Q_VIEW, index=False, encoding='utf-8-sig')

print()
print('[Phase 2] summary files saved')
print('  ops_group:', PHASE2_OPS_GROUP)
print('  ops_view :', PHASE2_OPS_VIEW)
print('  meta:', summary_out)

[Phase 2] quality valid rows (reloaded): 294 / 300
[Phase 2] quality summary by group
                config    group     kw_v2  faithfulness  context_recall
0    A_single_pipeline  central  0.258013      0.676282        0.326923
1    A_single_pipeline  extreme  0.300694      0.741667        0.350000
2  B_rule_single_route  central  0.158440      0.860256        0.337607
3  B_rule_single_route  extreme  0.078472      0.933333        0.375000
4   C_rule_multi_route  central  0.177671      0.836538        0.382479
5   C_rule_multi_route  extreme  0.078472      0.908333        0.325000

[Phase 2] quality summary views
                config     kw_v2  faithfulness  context_recall  \
0    A_single_pipeline  0.266723      0.689626        0.331633   
1  B_rule_single_route  0.142120      0.875170        0.345238   
2   C_rule_multi_route  0.157426      0.851190        0.370748   
3    A_single_pipeline  0.279354      0.708974        0.338462   
4  B_rule_single_route  0.118456      0.896795 

## Phase 2 결과 해석

### 실행/집계 상태 요약

| 항목 | 값 | 해석 |
|------|----|------|
| 실행 계획 행 수 | 300 | 100문서 × 3개 config(A/B/C) |
| 품질 유효 행 | 294/300 | 대부분 키에서 KW/faithfulness/context_recall 계산 완료 |
| 답변 동일률(A/B/C) | 43/100 (43.0%) | config별 출력 분리가 실제로 발생 |
| 컨텍스트 동일률(A/B/C) | 29/98 (29.6%) | retrieval 결과도 config별 차이가 형성됨 |
| quality/ops 결과 파일 | 생성 완료 | `exp09_phase2_quality_*.csv`, `exp09_phase2_ops_*.csv` 정상 생성 |

### 품질 지표 비교 (Phase 2 결과)

| View | A_single | B_rule_single | C_rule_multi |
|------|----------|---------------|--------------|
| **KW_v2 (overall)** | **0.2667** | 0.1421 | 0.1574 |
| **Faithfulness (overall)** | 0.6896 | **0.8752** | 0.8512 |
| **Context Recall (overall)** | 0.3316 | 0.3452 | **0.3707** |
| **KW_v2 (worst-group)** | **0.2580** | 0.0785 | 0.0785 |
| **Faithfulness (worst-group)** | 0.6763 | **0.8603** | 0.8365 |
| **Context Recall (worst-group)** | 0.3269 | **0.3376** | 0.3250 |

### 운영 지표 비교 (dry-run)

| View | A_single | B_rule_single | C_rule_multi |
|------|----------|---------------|--------------|
| ingestion_success (overall) | **0.7403** | 0.7110 | 0.6985 |
| timeout_rate (overall) | **0.2597** | 0.2890 | 0.3015 |
| fallback_rate (overall) | **0.0723** | 0.0758 | 0.1019 |
| p95_latency_sec (overall) | **36.83** | 39.81 | 41.31 |
| p95_latency_sec (worst-group) | **42.80** | 46.25 | 46.25 |

### 핵심 해석

1. **품질-운영 트레이드오프가 명확히 분리됨**
- A는 KW_v2가 가장 높고 운영 안정성도 가장 좋음.
- B/C는 Faithfulness/Context Recall이 높지만 운영 비용(시간/timeout/fallback)이 증가.

2. **C는 Recall 이점이 있으나 운영 부담이 큼**
- Overall CR은 C가 최고(0.3707)지만 timeout/p95가 가장 나쁨.
- worst-group CR은 B가 소폭 우세(0.3376).

3. **현재 데이터 기준 잠정 선택**
- 운영 안정성 우선이면 **A_single_pipeline**.
- 근거성(Faithfulness)과 CR 우선이면 **B_rule_single_route**가 C 대비 비용 효율적.

### Phase 3 시사점

1. Phase 3(2x2)에서는 B를 기준선으로 두고 table/image 경로의 추가 기여를 분해한다.
2. 최종 의사결정은 single metric이 아니라 `KW_v2 + CR + timeout/p95` 동시 기준으로 한다.
3. 보고서에는 본 Phase 2의 answer/context가 실험용 자동 생성 경로임을 명시한다.

---
## Phase 3: Table/Image 2x2 Ablation (Subgroup)
**실험**: table-aware/OCR 기여도를 subgroup 단위로 분해
- factor A: table-aware ON/OFF
- factor B: OCR ON/OFF
- 중앙군/극단군 각각에서 효과 비교 (전체 평균만 보고 결론 내리지 않음)


In [ ]:
# ============================================================
# Phase 3: Table/Image 2x2 분석 (subgroup)
# ============================================================
PHASE2_PLAN_PATH = OUT_DIR / 'exp09_phase2_plan.csv'
PHASE2_METRICS_PATH = OUT_DIR / 'exp09_phase2_metrics.csv'

if not PHASE2_PLAN_PATH.exists() or not PHASE2_METRICS_PATH.exists():
    raise FileNotFoundError('Phase2 결과 파일이 없습니다. Phase2를 먼저 실행하세요.')

plan_df = pd.read_csv(PHASE2_PLAN_PATH)
metrics_df = pd.read_csv(PHASE2_METRICS_PATH)

# phase2_plan에는 run_id가 없을 수 있어 보정
if 'run_id' not in plan_df.columns:
    plan_df['run_id'] = 1
if 'run_id' not in metrics_df.columns:
    metrics_df['run_id'] = 1

join_keys = ['config', 'run_id', 'file', 'group']
need_cols = join_keys + ['effective_route', 'effective_mode']
missing_plan_cols = [c for c in need_cols if c not in plan_df.columns]
if missing_plan_cols:
    raise KeyError(f'phase2_plan.csv에 필요한 컬럼 누락: {missing_plan_cols}')

# quality 유효행만 사용
m = metrics_df.copy()
valid_mask = m[['kw_v2', 'faithfulness', 'context_recall']].notna().all(axis=1)
m_valid = m[valid_mask].copy()
print('[Phase 3] valid quality rows:', len(m_valid), '/', len(m))

merged = m_valid.merge(plan_df[need_cols].drop_duplicates(join_keys), on=join_keys, how='left')

# 2x2 factor: route 기반 관측치 분류 (실험형 2x2가 아닌 observational 2x2)
merged['table_aware'] = merged['effective_route'].isin(['table_aware', 'text+table']).astype(int)
merged['ocr'] = merged['effective_route'].isin(['image_aware', 'text+image']).astype(int)
merged['cell'] = merged['table_aware'].astype(str) + merged['ocr'].astype(str)

# subgroup x 2x2 평균
agg_cols = ['kw_v2', 'faithfulness', 'context_recall']
grid = merged.groupby(['group', 'table_aware', 'ocr'])[agg_cols].mean().reset_index()
counts = merged.groupby(['group', 'table_aware', 'ocr']).size().reset_index(name='n')
grid = grid.merge(counts, on=['group', 'table_aware', 'ocr'], how='left')

print('\n[Phase 3] subgroup x 2x2 mean')
print(grid)

# config-level 매핑 참고 출력
cfg_map = merged.groupby('config')[['table_aware','ocr']].agg(['mean','min','max'])
print('\n[Phase 3] config->factor mapping (observational)')
print(cfg_map)


def effect_table(df_group, metric):
    # image 고정 조건에서 table 효과 평균
    vals = []
    for o in [0, 1]:
        a = df_group[(df_group['table_aware'] == 1) & (df_group['ocr'] == o)][metric]
        b = df_group[(df_group['table_aware'] == 0) & (df_group['ocr'] == o)][metric]
        if len(a) > 0 and len(b) > 0:
            vals.append(float(a.mean() - b.mean()))
    return float(np.mean(vals)) if len(vals) > 0 else np.nan


def effect_ocr(df_group, metric):
    # table 고정 조건에서 ocr 효과 평균
    vals = []
    for t in [0, 1]:
        a = df_group[(df_group['table_aware'] == t) & (df_group['ocr'] == 1)][metric]
        b = df_group[(df_group['table_aware'] == t) & (df_group['ocr'] == 0)][metric]
        if len(a) > 0 and len(b) > 0:
            vals.append(float(a.mean() - b.mean()))
    return float(np.mean(vals)) if len(vals) > 0 else np.nan


def interaction(df_group, metric):
    # (T1I1 - T0I1) - (T1I0 - T0I0)
    def m(t, o):
        x = df_group[(df_group['table_aware'] == t) & (df_group['ocr'] == o)][metric]
        return float(x.mean()) if len(x) > 0 else np.nan
    m11, m01, m10, m00 = m(1,1), m(0,1), m(1,0), m(0,0)
    if any(pd.isna(v) for v in [m11, m01, m10, m00]):
        return np.nan
    return float((m11 - m01) - (m10 - m00))


rows = []
for grp in sorted(merged['group'].dropna().unique().tolist()):
    gdf = merged[merged['group'] == grp].copy()
    for metric in agg_cols:
        rows.append({
            'group': grp,
            'metric': metric,
            'table_effect': effect_table(gdf, metric),
            'ocr_effect': effect_ocr(gdf, metric),
            'interaction': interaction(gdf, metric),
            'n': int(len(gdf)),
        })

phase3_effects = pd.DataFrame(rows)
print('\n[Phase 3] effects by subgroup')
print(phase3_effects)

# 저장
PHASE3_GRID_PATH = OUT_DIR / 'exp09_phase3_2x2_group.csv'
PHASE3_EFFECTS_PATH = OUT_DIR / 'exp09_phase3_effects.csv'

grid.to_csv(PHASE3_GRID_PATH, index=False, encoding='utf-8-sig')
phase3_effects.to_csv(PHASE3_EFFECTS_PATH, index=False, encoding='utf-8-sig')

print('\n[Phase 3] saved files')
print('  grid   :', PHASE3_GRID_PATH)
print('  effects:', PHASE3_EFFECTS_PATH)

[Phase 3] valid quality rows: 294 / 300

[Phase 3] subgroup x 2x2 mean
     group  table_aware  ocr     kw_v2  faithfulness  context_recall    n
0  central            0    0  0.244792      0.707721        0.323529  136
1  central            0    1  0.000000      1.000000        0.000000    2
2  central            1    0  0.135938      0.904687        0.392361   96
3  extreme            0    0  0.300694      0.741667        0.350000   20
4  extreme            0    1  0.000000      1.000000        0.000000    2
5  extreme            1    0  0.082602      0.916667        0.368421   38

[Phase 3] config->factor mapping (observational)
                    table_aware               ocr        
                           mean min max      mean min max
config                                                   
A_single_pipeline      0.000000   0   0  0.000000   0   0
B_rule_single_route    0.683673   0   1  0.020408   0   1
C_rule_multi_route     0.683673   0   1  0.020408   0   1

[Phase 3] ef

## Phase 3 결과 해석

### 2x2 분석 요약 (subgroup)

| group | table_aware | ocr | KW_v2 | Faithfulness | Context Recall | n |
|------|-------------|-----|------|--------------|----------------|---|
| central | 0 | 0 | 0.2448 | 0.7077 | 0.3235 | 136 |
| central | 1 | 0 | 0.1359 | 0.9047 | 0.3924 | 96 |
| central | 0 | 1 | 0.0000 | 1.0000 | 0.0000 | 2 |
| extreme | 0 | 0 | 0.3007 | 0.7417 | 0.3500 | 20 |
| extreme | 1 | 0 | 0.0826 | 0.9167 | 0.3684 | 38 |
| extreme | 0 | 1 | 0.0000 | 1.0000 | 0.0000 | 2 |

### 효과 추정 결과 (observational)

| group | metric | table_effect | ocr_effect | 해석 |
|------|--------|-------------:|-----------:|------|
| central | KW_v2 | -0.1089 | -0.2448 | table/ocr 경로에서 키워드 재현율 하락 |
| central | Faithfulness | +0.1970 | +0.2923 | 근거 일치성은 table/ocr 경로에서 상승 |
| central | Context Recall | +0.0688 | -0.3235 | table은 소폭 개선, ocr은 샘플 부족으로 불안정 |
| extreme | KW_v2 | -0.2181 | -0.3007 | 극단군에서 KW_v2 하락폭 더 큼 |
| extreme | Faithfulness | +0.1750 | +0.2583 | 극단군도 근거 일치성은 상승 |
| extreme | Context Recall | +0.0184 | -0.3500 | table 효과는 미미, ocr은 해석 보류 필요 |

### 핵심 발견

**1. table_aware는 “KW_v2 하락 vs Faithfulness 상승”의 트레이드오프가 일관적**
- central/extreme 모두에서 table_effect가 KW_v2 음수, Faithfulness 양수로 동일 패턴.
- 즉 table 경로는 표현 재현(키워드)보다 근거 정합(faithfulness)에 유리한 방향으로 작동.

**2. OCR 효과는 현재 데이터에서 결론 보류가 타당**
- `ocr=1` 표본이 각 그룹 2건(n=2)으로 매우 작음.
- 현재 관측치만으로 OCR 경로의 일반 효과를 확정하면 과해석 위험이 큼.

**3. interaction이 NaN인 것은 데이터 구조상 정상**
- 2x2 네 칸이 그룹별로 모두 채워지지 않아 (특히 ocr=1 희소) 상호작용항 계산이 불가.
- 이는 코드 오류가 아니라 표본 커버리지 부족 문제.

### Phase 4 시사점

1. **table_aware 기본 적용 여부는 단일 지표가 아니라 복합 기준으로 결정**
- KW_v2 하락을 감수할지, Faithfulness/CR 개선을 우선할지 정책 결정 필요.

2. **OCR 경로는 확대 검증 후 판단**
- 최소한 subgroup별 `ocr=1` 표본 수를 늘린 뒤 재평가.

3. **최종 선택 기준 유지**
- `overall + worst-group + ops(timeout/p95)`를 함께 보고 의사결정.

짧게 덧붙이면, 현재 Phase 3는 “코드 문제”보다 “표본 불균형(ocr=1 부족)”이 해석 한계의 핵심입니다.

---
## Phase 4: 운영 안정성 검증 + 최종 의사결정
**실험**: 품질 지표와 운영 지표를 함께 평가하여 최종 아키텍처 선택
- Ops KPI: ingestion_success, timeout_rate, fallback_rate, p95 latency
- Decision Rule: 품질 기준 충족 + worst-group 하한 + 운영 안정성 기준 동시 만족


In [16]:
# ============================================================
# Phase 4: Decision Rule Evaluation (from Phase2/3 outputs)
# ============================================================
DECISION_RULE = {
    'quality_floor': {
        'kw_v2': 0.75,
        'faithfulness': 0.90,
        'context_recall': 0.88,
    },
    'worst_group_floor': {
        'kw_v2': 0.68,
        'faithfulness': 0.86,
        'context_recall': 0.80,
    },
    'ops_ceiling': {
        'timeout_rate': 0.05,
        'fallback_rate': 0.20,
        'p95_latency_sec': 120,
    }
}

Q_PATH = OUT_DIR / 'exp09_phase2_quality_view.csv'
O_PATH = OUT_DIR / 'exp09_phase2_ops_view.csv'

print('[Phase 4] decision rule loaded')
print(json.dumps(DECISION_RULE, indent=2, ensure_ascii=False))

if not Q_PATH.exists() or not O_PATH.exists():
    raise FileNotFoundError('Phase2 quality/ops 결과 파일이 없습니다. Phase2 집계를 먼저 실행하세요.')

qv = pd.read_csv(Q_PATH)
ov = pd.read_csv(O_PATH)

# overall + worst_group만 사용
q_overall = qv[qv['view'] == 'overall_mean'].copy()
q_worst = qv[qv['view'] == 'worst_group'].copy()
o_overall = ov[ov['view'] == 'overall_mean'].copy()
o_worst = ov[ov['view'] == 'worst_group'].copy()

q_overall = q_overall.rename(columns={
    'kw_v2': 'kw_v2_overall',
    'faithfulness': 'faithfulness_overall',
    'context_recall': 'context_recall_overall',
})[['config', 'kw_v2_overall', 'faithfulness_overall', 'context_recall_overall']]

q_worst = q_worst.rename(columns={
    'kw_v2': 'kw_v2_worst',
    'faithfulness': 'faithfulness_worst',
    'context_recall': 'context_recall_worst',
})[['config', 'kw_v2_worst', 'faithfulness_worst', 'context_recall_worst']]

o_overall = o_overall.rename(columns={
    'timeout_rate': 'timeout_overall',
    'fallback_rate': 'fallback_overall',
    'p95_latency_sec': 'p95_overall',
})[['config', 'timeout_overall', 'fallback_overall', 'p95_overall']]

o_worst = o_worst.rename(columns={
    'timeout_rate': 'timeout_worst',
    'fallback_rate': 'fallback_worst',
    'p95_latency_sec': 'p95_worst',
})[['config', 'timeout_worst', 'fallback_worst', 'p95_worst']]

decision_df = q_overall.merge(q_worst, on='config', how='inner')     .merge(o_overall, on='config', how='inner')     .merge(o_worst, on='config', how='inner')

# hard gate
qf = DECISION_RULE['quality_floor']
wf = DECISION_RULE['worst_group_floor']
oc = DECISION_RULE['ops_ceiling']

decision_df['pass_quality_overall'] = (
    (decision_df['kw_v2_overall'] >= qf['kw_v2']) &
    (decision_df['faithfulness_overall'] >= qf['faithfulness']) &
    (decision_df['context_recall_overall'] >= qf['context_recall'])
)

decision_df['pass_quality_worst'] = (
    (decision_df['kw_v2_worst'] >= wf['kw_v2']) &
    (decision_df['faithfulness_worst'] >= wf['faithfulness']) &
    (decision_df['context_recall_worst'] >= wf['context_recall'])
)

decision_df['pass_ops_overall'] = (
    (decision_df['timeout_overall'] <= oc['timeout_rate']) &
    (decision_df['fallback_overall'] <= oc['fallback_rate']) &
    (decision_df['p95_overall'] <= oc['p95_latency_sec'])
)

decision_df['pass_ops_worst'] = (
    (decision_df['timeout_worst'] <= oc['timeout_rate']) &
    (decision_df['fallback_worst'] <= oc['fallback_rate']) &
    (decision_df['p95_worst'] <= oc['p95_latency_sec'])
)

decision_df['pass_all'] = (
    decision_df['pass_quality_overall'] &
    decision_df['pass_quality_worst'] &
    decision_df['pass_ops_overall'] &
    decision_df['pass_ops_worst']
)

# soft score (gate 미충족 시 상대 비교용)
# 가중치: quality 70%, ops 30%
decision_df['quality_score'] = (
    0.35 * decision_df['kw_v2_overall'] +
    0.20 * decision_df['faithfulness_overall'] +
    0.15 * decision_df['context_recall_overall']
)

decision_df['ops_score'] = (
    0.15 * (1 - decision_df['timeout_overall']) +
    0.10 * (1 - decision_df['fallback_overall']) +
    0.05 * (1 - (decision_df['p95_overall'] / max(1.0, decision_df['p95_overall'].max())))
)

decision_df['total_score'] = decision_df['quality_score'] + decision_df['ops_score']

# 선택 규칙: pass_all 우선, 없으면 total_score 최대
if decision_df['pass_all'].any():
    recommended = decision_df[decision_df['pass_all']].sort_values('total_score', ascending=False).iloc[0]
    reason = 'hard gate 충족 후보 중 total_score 최대'
else:
    recommended = decision_df.sort_values('total_score', ascending=False).iloc[0]
    reason = 'hard gate 미충족 -> soft ranking(total_score) 사용'

PHASE4_DECISION_PATH = OUT_DIR / 'exp09_phase4_decision.csv'
decision_df.to_csv(PHASE4_DECISION_PATH, index=False, encoding='utf-8-sig')

print('\n[Phase 4] decision table')
print(decision_df[['config','pass_all','quality_score','ops_score','total_score']])
print('\n[Phase 4] recommended config:', recommended['config'])
print('[Phase 4] reason:', reason)
print('[Phase 4] saved:', PHASE4_DECISION_PATH)

[Phase 4] decision rule loaded
{
  "quality_floor": {
    "kw_v2": 0.75,
    "faithfulness": 0.9,
    "context_recall": 0.88
  },
  "worst_group_floor": {
    "kw_v2": 0.68,
    "faithfulness": 0.86,
    "context_recall": 0.8
  },
  "ops_ceiling": {
    "timeout_rate": 0.05,
    "fallback_rate": 0.2,
    "p95_latency_sec": 120
  }
}

[Phase 4] decision table
                config  pass_all  quality_score  ops_score  total_score
0    A_single_pipeline     False       0.281023   0.209227     0.490250
1  B_rule_single_route     False       0.276562   0.200889     0.477450
2   C_rule_multi_route     False       0.280950   0.194584     0.475533

[Phase 4] recommended config: A_single_pipeline
[Phase 4] reason: hard gate 미충족 -> soft ranking(total_score) 사용
[Phase 4] saved: ..\data\experiments\exp09_phase4_decision.csv


## Phase 4 결과 해석

### 의사결정 규칙 적용 결과

| Config | pass_all | quality_score | ops_score | total_score | 판정 |
|--------|----------|---------------|-----------|-------------|------|
| A_single_pipeline | False | 0.2810 | 0.2092 | **0.4903** | 1순위 (soft) |
| B_rule_single_route | False | 0.2766 | 0.2009 | 0.4775 | 2순위 |
| C_rule_multi_route | False | 0.2810 | 0.1946 | 0.4755 | 3순위 |

- **Hard gate 충족 config 없음** (`pass_all=False` 전원)
- 따라서 Phase 4는 `total_score` 기반 soft ranking으로 선택
- **최종 추천 config: `A_single_pipeline`**

### 왜 hard gate를 통과하지 못했는가

현재 규칙 임계값:
- quality_floor: `KW_v2 >= 0.75`, `Faithfulness >= 0.90`, `CR >= 0.88`
- worst_group_floor: `KW_v2 >= 0.68`, `Faithfulness >= 0.86`, `CR >= 0.80`
- ops_ceiling: `timeout <= 0.05`, `fallback <= 0.20`, `p95 <= 120`

관측치(Phase 2/3)와 비교 시:
- KW_v2, CR이 floor 대비 큰 폭으로 미달
- timeout도 ceiling(0.05) 대비 높음
- 즉, 현재 threshold는 본 실험 스케일에서 매우 공격적인 기준으로 확인됨

### 해석

1. **현 시점 운영 관점 최적은 A_single_pipeline**
- ops_score가 가장 높고 total_score 1위
- worst-group 운영 리스크도 B/C 대비 낮은 편

2. **품질 우선 전략(B/C)의 이점은 있으나 운영 비용 증가**
- B/C는 faithfulness·CR 측면에서 강점이 있으나
- timeout/fallback/p95 악화로 hard gate 관점에서 손실

3. **다음 단계는 임계값 보정 + 분기 전략 고도화**
- 현재 hard gate는 “목표치” 성격으로 유지하되, 실측 기반 단계적 목표로 보정 필요
- A를 baseline으로 두고 B/C의 품질 이득을 저비용으로 이식하는 방향이 현실적

In [17]:
# ============================================================
# Phase 4: 보고서 저장
# ============================================================
PHASE2_Q_PATH = OUT_DIR / 'exp09_phase2_quality_view.csv'
PHASE2_O_PATH = OUT_DIR / 'exp09_phase2_ops_view.csv'
PHASE4_D_PATH = OUT_DIR / 'exp09_phase4_decision.csv'

report = {
    'experiment': 'EXP09: Generalization Verification (Routing + Worst-group)',
    'timestamp': datetime.now().isoformat(),
    'dataset': {
        'total_docs': int(len(df)),
        'central_docs': int((fp['group'] == 'central').sum()) if 'group' in fp.columns else None,
        'extreme_docs': int((fp['group'] == 'extreme').sum()) if 'group' in fp.columns else None,
    },
    'routing': {
        'route_distribution': phase1_df['route'].value_counts().to_dict() if 'phase1_df' in locals() else {},
        'route_mode_distribution': phase1_df['route_mode'].value_counts().to_dict() if 'phase1_df' in locals() else {},
    },
    'decision_rule': DECISION_RULE,
}

if PHASE2_Q_PATH.exists():
    qv = pd.read_csv(PHASE2_Q_PATH)
    report['phase2_quality_view'] = qv.to_dict(orient='records')

if PHASE2_O_PATH.exists():
    ov = pd.read_csv(PHASE2_O_PATH)
    report['phase2_ops_view'] = ov.to_dict(orient='records')

if PHASE4_D_PATH.exists():
    dv = pd.read_csv(PHASE4_D_PATH)
    report['phase4_decision_table'] = dv.to_dict(orient='records')
    if len(dv) > 0:
        report['phase4_recommended'] = dv.sort_values('total_score', ascending=False).iloc[0].to_dict()

with open(OUT_REPORT, 'w', encoding='utf-8') as f:
    json.dump(report, f, indent=2, ensure_ascii=False)

if 'phase1_df' in locals():
    phase1_df.to_csv(OUT_RESULTS, index=False, encoding='utf-8-sig')

print(f'  Report: {OUT_REPORT}')
print(f'  CSV: {OUT_RESULTS}')
if PHASE4_D_PATH.exists():
    print(f'  Decision: {PHASE4_D_PATH}')



  Report: ..\data\experiments\exp09_report.json
  CSV: ..\data\experiments\exp09_results.csv
  Decision: ..\data\experiments\exp09_phase4_decision.csv


---

## 결론

-  EXP09 결과, 동일 질문/정답 조건에서 A/B/C별 출력을 분리해 비교한 결과 hard gate를 충족하는 설정은 없었고, soft ranking 기준으로
-  `A_single_pipeline`이 최종 1순위로 선정되었다.
-  `B_rule_single_route`, `C_rule_multi_route`는 Faithfulness/Context Recall 측면의 이점이 관측되었으나, 운영 지표(timeout/fallback/p95latency) 부담이 커 총점에서 A를 넘지 못했다.
-  따라서 현 단계의 운영 기준선은 A로 확정하고, 후속 개선은 B/C의 품질 이득을 A의 운영 안정성을 해치지 않는 범위에서 부분 이식하는 방향이 타당하다.
-  또한 현재 hard gate 임계값은 실측 분포 대비 매우 공격적이므로, 다음 실험에서는 단계형 임계값(현실 구간 → 목표 구간)으로 조정해 의사결정의 실효성을 높인다.
-  최종적으로 EXP09는 “단일 최적값”보다 “품질-운영 균형을 갖춘 아키텍처 선택”이 일반화 성능에 더 중요함을 확인했다.